In [114]:
run irw_gtfs.py

In [115]:
schedule = load_schedule('data/gtfs/2015_07_15_06_00_02.zip')

**Some basic research trying to understand the concept of a service in IRW gtfs**

In [116]:
# are start_date and end_date for all services?
multiple_dates = [service for service in schedule.services.values() if service.start_date != service.end_date]
print(len(multiple_dates))
# result is 0

0


In [117]:
# different arrival and depature times on any stop?
diff = 0
same = 0
for trip in schedule.trips:
    l = len([stop for stop in trip.stops if stop.arrival_time != stop.departure_time])
    diff += l
    same += len(trip.stops) - l
print("same arrival and departure", same, "different arrival and depature", diff)
# same arrival and departure 250938 different arrival and depature 16794

same arrival and departure 250938 different arrival and depature 16794


In [118]:
trips_with_diff = []
for trip in schedule.trips:
    diff = [stop for stop in trip.stops if stop.arrival_time != stop.departure_time]
    if len(diff) > 0:
        trips_with_diff.append(trip)
print(len(trips_with_diff))
trips_with_diff[0].stops

16794


[StopTime(trip_id='010815_07001', arrival_time='23:11:00', departure_time='23:11:00', stop=Stop(stop_id='1600', stop_name='Nahariyya'), stop_sequence='1'),
 StopTime(trip_id='010815_07001', arrival_time='23:19:00', departure_time='23:19:00', stop=Stop(stop_id='1500', stop_name='Akko'), stop_sequence='2'),
 StopTime(trip_id='010815_07001', arrival_time='23:29:00', departure_time='23:29:00', stop=Stop(stop_id='800', stop_name='Kiryat Motzkin'), stop_sequence='3'),
 StopTime(trip_id='010815_07001', arrival_time='23:43:00', departure_time='23:43:00', stop=Stop(stop_id='2100', stop_name='Haifa Center HaShmona'), stop_sequence='4'),
 StopTime(trip_id='010815_07001', arrival_time='23:52:00', departure_time='23:52:00', stop=Stop(stop_id='2300', stop_name="Haifa Hof HaKarmel (Razi'el)"), stop_sequence='5'),
 StopTime(trip_id='010815_07001', arrival_time='24:12:00', departure_time='24:12:00', stop=Stop(stop_id='2800', stop_name='Binyamina'), stop_sequence='6'),
 StopTime(trip_id='010815_07001', 

In [119]:
stops_with_diff = []
for trip in schedule.trips:
    stops_with_diff += [stop for stop in trip.stops if stop.arrival_time != stop.departure_time]
print(len(stops_with_diff))
print(set(stop.stop for stop in stops_with_diff))
# only stops in Tel Aviv Center have difference 

16794
{Stop(stop_id='3700', stop_name='Tel Aviv Center - Savidor')}


In [120]:
# what's the length of the stop?
def parse_time(s):
    return int(s[:2]) * 60 * 60 + int(s[3:5]) * 60 + int(s[6:])

def time_at_station(stop):
    return parse_time(stop.departure_time) - parse_time(stop.arrival_time)

from collections import Counter 
c = Counter()
for stop in stops_with_diff:
    c[time_at_station(stop)] += 1
print(c)
# Counter({180: 11970, 120: 4824})

Counter({180: 11970, 120: 4824})


In [121]:
# is time in Tel-Aviv center ever 0? 
stops_at_tlv_center_with_length_zero = []
for trip in schedule.trips:
    diff = [stop for stop in trip.stops if  stop.stop.stop_id == '3700' and stop.arrival_time == stop.departure_time]
    if len(diff) > 0:
        stops_at_tlv_center_with_length_zero.append(trip)
print(len(stops_at_tlv_center_with_length_zero))
stops_at_tlv_center_with_length_zero[0]
# yes that's happens as well in 2439 trips

2439


Trip(route=Route(route_id='60', route_long_name='Nahariyya - Ben Gurion Airport'), service=Service(service_id='010815_0000001', day=5, start_date='20150801', end_date='20150801'), trip_id='010815_07018', trip_headsign='Tel Aviv Center - Savidor', direction_id='0', stops=[StopTime(trip_id='010815_07018', arrival_time='20:40:00', departure_time='20:40:00', stop=Stop(stop_id='7320', stop_name="Be'er Sheva Center"), stop_sequence='1'), StopTime(trip_id='010815_07018', arrival_time='20:47:00', departure_time='20:47:00', stop=Stop(stop_id='7300', stop_name="Be'er Sheva North University"), stop_sequence='2'), StopTime(trip_id='010815_07018', arrival_time='20:58:00', departure_time='20:58:00', stop=Stop(stop_id='8550', stop_name='Lehavim - Rahat'), stop_sequence='3'), StopTime(trip_id='010815_07018', arrival_time='21:15:00', departure_time='21:15:00', stop=Stop(stop_id='7000', stop_name='Kiryat Gat'), stop_sequence='4'), StopTime(trip_id='010815_07018', arrival_time='21:40:00', departure_time=

In [122]:
# look for same trip in subsequent days? At least Mon-Wed should be rather similar? 
trip0 = schedule.trips[0]
shouldbe_same_service = [trip for trip in schedule.trips if trip.route.route_id == trip0.route.route_id and trip.stops[0].departure_time == trip0.stops[0].departure_time]
print("number of trips", len(shouldbe_save_service))
print("trip dates", list(sorted(trip.service.start_date for trip in shouldbe_save_service)))


number of trips 54
trip dates ['20150714', '20150715', '20150716', '20150718', '20150719', '20150720', '20150721', '20150722', '20150723', '20150725', '20150726', '20150727', '20150728', '20150729', '20150730', '20150801', '20150802', '20150803', '20150804', '20150805', '20150806', '20150808', '20150809', '20150810', '20150811', '20150812', '20150813', '20150815', '20150816', '20150817', '20150818', '20150819', '20150820', '20150822', '20150823', '20150824', '20150825', '20150826', '20150827', '20150829', '20150830', '20150831', '20150901', '20150902', '20150903', '20150905', '20150906', '20150907', '20150908', '20150909', '20150910', '20150912', '20150913', '20150914']


In [123]:
# are trip really identical in all stations? 
def same_stations(trip1, trip2):
    for s1, s2 in zip(trip1.stops, trip2.stops):
        # StopTime = namedtuple('StopTime', 'trip_id arrival_time departure_time stop stop_sequence')
        if s1.arrival_time != s2.arrival_time or s1.departure_time != s2.departure_time or s1.stop.stop_id != s2.stop.stop_id or s1.stop_sequence != s2.stop_sequence:
            return False 
    return True

have_same_stops = [trip for trip in shouldbe_same_service if same_stations(trip, trip0)]        
print(len(have_same_stops))     
# 54. Indeed they are all the same service. 

54


In [124]:
trip0

Trip(route=Route(route_id='60', route_long_name='Nahariyya - Ben Gurion Airport'), service=Service(service_id='010815_0000001', day=5, start_date='20150801', end_date='20150801'), trip_id='010815_07001', trip_headsign='Ben Gurion Airport', direction_id='1', stops=[StopTime(trip_id='010815_07001', arrival_time='23:11:00', departure_time='23:11:00', stop=Stop(stop_id='1600', stop_name='Nahariyya'), stop_sequence='1'), StopTime(trip_id='010815_07001', arrival_time='23:19:00', departure_time='23:19:00', stop=Stop(stop_id='1500', stop_name='Akko'), stop_sequence='2'), StopTime(trip_id='010815_07001', arrival_time='23:29:00', departure_time='23:29:00', stop=Stop(stop_id='800', stop_name='Kiryat Motzkin'), stop_sequence='3'), StopTime(trip_id='010815_07001', arrival_time='23:43:00', departure_time='23:43:00', stop=Stop(stop_id='2100', stop_name='Haifa Center HaShmona'), stop_sequence='4'), StopTime(trip_id='010815_07001', arrival_time='23:52:00', departure_time='23:52:00', stop=Stop(stop_id='

**conculsion:** IRW just doesn't use service to its full power of normalisation.

Instead, service just gives you the date of the trip. 

In [125]:
# lets see now what dates are convered by this file
dates = sorted(set(trip.service.start_date for trip in schedule.trips))
print(list(dates))

['20150714', '20150715', '20150716', '20150717', '20150718', '20150719', '20150720', '20150721', '20150722', '20150723', '20150724', '20150725', '20150726', '20150727', '20150728', '20150729', '20150730', '20150731', '20150801', '20150802', '20150803', '20150804', '20150805', '20150806', '20150807', '20150808', '20150809', '20150810', '20150811', '20150812', '20150813', '20150814', '20150815', '20150816', '20150817', '20150818', '20150819', '20150820', '20150821', '20150822', '20150823', '20150824', '20150825', '20150826', '20150827', '20150828', '20150829', '20150830', '20150831', '20150901', '20150902', '20150903', '20150904', '20150905', '20150906', '20150907', '20150908', '20150909', '20150910', '20150911', '20150912', '20150913', '20150914']


This zip file contains all dates between 20150714 and 20150914 

In [126]:
# group_by is defined in irw_gtfs
def group_by_date(schedule):
    return group_by(schedule.trips, lambda trip: trip.service.start_date)

grouped_by_date = group_by_date(schedule)
for date in sorted(grouped_by_date.keys()):
    print(date, len(grouped_by_date[date]))



20150714 384
20150715 384
20150716 384
20150717 217
20150718 37
20150719 387
20150720 384
20150721 384
20150722 384
20150723 384
20150724 217
20150725 37
20150726 387
20150727 384
20150728 384
20150729 384
20150730 384
20150731 217
20150801 37
20150802 387
20150803 384
20150804 384
20150805 384
20150806 384
20150807 217
20150808 37
20150809 387
20150810 384
20150811 384
20150812 384
20150813 384
20150814 217
20150815 37
20150816 387
20150817 384
20150818 384
20150819 384
20150820 384
20150821 217
20150822 37
20150823 387
20150824 384
20150825 384
20150826 384
20150827 384
20150828 217
20150829 37
20150830 387
20150831 384
20150901 384
20150902 384
20150903 384
20150904 217
20150905 37
20150906 387
20150907 384
20150908 384
20150909 384
20150910 384
20150911 217
20150912 37
20150913 387
20150914 384


There's regularly 387 trains on Sunday, 384 Mon-Thur, 217 Fri and 37 Saturday

For every date in this gtfs, all trains are included.

Next question, do files always agree on overlapping days?

In [81]:
def agree(schedule1, schedule2):
    by_trip_id1 = {trip.trip_id: trip for trip in schedule1.trips}
    by_trip_id2 = {trip.trip_id: trip for trip in schedule2.trips}
    common_trips = set(by_trip_id1.keys()).intersection(by_trip_id2.keys())
    print(len(common_trips), len(common_trips)/len(schedule1.trips), len(common_trips)/len(schedule2.trips))
    for trip_id in common_trips:
        if by_trip_id1[trip_id] != by_trip_id2[trip_id]:
            return False
    return True

In [93]:
import os
s1 = schedule
for f in reversed(sorted([f for f in os.listdir("data/gtfs") if f.endswith('.zip')])):
    s2 = load(os.path.join('data/gtfs', f))
    print(f, agree(s1, s2))
    s1 = s2

19593 1.0 1.0
2015_07_15_06_00_02.zip True
19209 0.9804011636809065 0.9804011636809065
2015_07_14_06_00_02.zip True
19162 0.9780023477772674 0.9780023477772674
2015_07_13_06_00_02.zip True
19556 0.9981115704588374 0.9981115704588374
2015_07_12_06_00_03.zip True
19376 0.9889246159342623 1.0
2015_07_11_06_00_02.zip True
18992 0.9801816680429397 0.9693257796151686
2015_07_10_06_00_02.zip True
19209 0.9804011636809065 0.9804011636809065
2015_07_09_06_00_02.zip True
19209 0.9804011636809065 0.9804011636809065
2015_07_08_06_00_02.zip True
19209 0.9804011636809065 0.9804011636809065
2015_07_07_06_00_02.zip True
19206 0.9802480477721636 0.9802480477721636
2015_07_06_06_00_02.zip True
19556 0.9981115704588374 0.9981115704588374
2015_07_05_06_00_02.zip True
19376 0.9889246159342623 0.9889246159342623
2015_07_04_06_00_01.zip True
19209 0.9804011636809065 0.9803511278962947
2015_07_03_06_00_02.zip True
19210 0.9804021639277329 0.9803021024698918
2015_07_02_06_00_01.zip True
18827 0.960757297407634

BadZipFile: File is not a zip file

In [97]:
# find an example of a difference? 
def disagree_on(schedule1, schedule2):
    by_trip_id1 = {trip.trip_id: trip for trip in schedule1.trips}
    by_trip_id2 = {trip.trip_id: trip for trip in schedule2.trips}
    common_trips = set(by_trip_id1.keys()).intersection(by_trip_id2.keys())
    for trip_id in common_trips:
        if by_trip_id1[trip_id] != by_trip_id2[trip_id]:
            return (by_trip_id1[trip_id] , by_trip_id2[trip_id])

diff = disagree_on(load('data/gtfs/2015_06_21_06_00_02.zip'), load('data/gtfs/2015_06_22_06_00_02.zip'))

In [112]:
# where is the difference?
# route service trip_id trip_headsign direction_id stops
print(diff[0].stops[0], diff[1].stops[0])

StopTime(trip_id='010715_00053', arrival_time='23:07:00', departure_time='23:07:00', stop=Stop(stop_id='3700', stop_name='Tel Aviv Center - Savidor'), stop_sequence='1') StopTime(trip_id='010715_00053', arrival_time='23:03:00', departure_time='23:03:00', stop=Stop(stop_id='3600', stop_name='Tel Aviv - University'), stop_sequence='1')


It appears that on June 22 it was decided that this train, instead of leaving at 23:07 from Tel-Aviv Center, will leave at 23:03 from Tel-Aviv Univerisity **and not stop in Tel-Aviv Center** 

This change have persisted to the next gtfs version.

In [113]:
a = {1: 2}
b = {1: 3}
a.update(b)
a

{1: 3}